In [ ]:
import csv

class RowModel:
    def __init__(self, row, header):
        self.__dict__ = dict(zip(header, row))
        self.district = self.Valdistriktnamn.strip()
        self.party = self.Parti.strip()
        self.votes = int(self.Röster.strip())
        
    def __repr__(self):
        return f'{self.district} {self.party} {self.votes}'
    
def previewDict(something):
   for i in list(something.items())[0:3]:
    print(i)
        
        
data = list(csv.reader(open('data/as-utf-8.csv', encoding='utf-8'), delimiter = ';'))
data[0:2]

In [ ]:
instances = [RowModel(r, data[0]) for r in data[1:]]

In [ ]:
validVotesPerDistrict = [rm  for rm in instances if rm.party == 'Summa giltiga röster']
districtVotes = {}
for rm in validVotesPerDistrict:
    districtVotes[rm.district] = rm.votes
previewDict(districtVotes)

In [ ]:
partiesWeights = {
  'Moderaterna': 2,
  'Arbetarepartiet-Socialdemokraterna': 1,
  'Liberalerna (tidigare Folkpartiet)': 1,
  'Miljöpartiet de gröna': -1,
  'Sverigedemokraterna': -1,
  'Vänsterpartiet': -2,
  'Alternativ för Sverige' : -3,
  'Partiet Nyans': -3                
}

interestingParties = set(partiesWeights.keys())

interestingParties

In [ ]:
onlyInterestingParties = [rm for rm in instances if rm.party in interestingParties]
onlyInterestingParties[0:2]

In [ ]:
percentPerPartyPerDistrict = [(rm.party, 
                               rm.district,
                               100*rm.votes/districtVotes[rm.district]
                              )
                             for rm in onlyInterestingParties]
percentPerPartyPerDistrict[:2]

Now we have tuples: 

```Party | District | Percent in the district```

We need to create: 

```Party | District | Percent in the district | Index of this district if sorted by percentage for the party```

In [ ]:
partyDistrictPercentPosition = {}
for e in percentPerPartyPerDistrict:
    party = e[0]
    lst = partyDistrictPercentPosition[party] if party in partyDistrictPercentPosition else []
    lst.append((e[1], e[2]))
    partyDistrictPercentPosition[party] = lst

for k,v in partyDistrictPercentPosition.items():
    v.sort(key = lambda a: a[1], reverse = True)
    
previewDict(partyDistrictPercentPosition)

In [ ]:
# party district percentage position
finalTable = []
for party, perDistrictRandingDecs in partyDistrictPercentPosition.items():
    for position, districtAndPercentage in enumerate(perDistrictRandingDecs):
        finalTable.append((party, districtAndPercentage[0], districtAndPercentage[1], position))
        
finalTable[0:2]

To create ranking we need to know:

For each district 
what party 
got what position

then we make up a score from it

In [ ]:
districtToPartyRanking = {}
for e in finalTable:
    district = e[1]
    party = e[0]
    percent = e[2]
    ranking = e[3]
    
    if district not in districtToPartyRanking:
        districtToPartyRanking[district] = {}

    districtToPartyRanking[district][party] = (ranking, percent)
    
previewDict(districtToPartyRanking)

In [ ]:
def scoreByRank(district, ranks):
    score = 0;
    for party, weight in partiesWeights.items():
        r = ranks[party][0] if party in ranks else len(districtToPartyRanking) - 1
        score += (len(districtToPartyRanking) - r)*partiesWeights[party]
    
    return score

def scoreByPercentage(district, ranks):
    score = 0;
    for party, weight in partiesWeights.items():
        p = ranks[party][1] if party in ranks else 0
        score += p*partiesWeights[party]
    return score
    

districtToScore = []
for district, ranks in districtToPartyRanking.items():
    score = scoreByPercentage(district, ranks)
    totalVotes = districtVotes[district]
    districtToScore.append((district, score, ranks, totalVotes))
    
districtToScore.sort(reverse = True, key = lambda a: a[1])

print(f'total districts = {len(districtToScore)}')

from pprint import pprint

for index, r in enumerate(districtToScore):
    print(f'{index}')
    pprint(r)
    print()
    